**Задание 1**  
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [8]:
import numpy as np
import pandas as pd

def is_sorted_by_userid(filename):
    previous = None

    with open(filename) as f:
        header = f.readline()

        for line in f:
            line = line.strip().split(',')

            user_id = line[0]

            if previous is None:
                previous = user_id

            if previous > user_id:
                return False
            
    return True

def solve_wo_pandas(filename):
    previous = None
    timestamps = []
    result = {}

    with open(filename) as f:
        header = f.readline()

        for line in f:
            line = line.strip().split(',')

            user_id = line[0]
            timestamp = line[3]

            if previous is None:
                previous = user_id
            else:
                if user_id == previous:
                    timestamps.append(int(timestamp))
                else:
                    if len(timestamps) > 100:
                        result[previous] = max(timestamps) - min(timestamps)
                    timestamps = []
                    previous = user_id
     
    if previous is not None:
        result[previous] = max(timestamps) - min(timestamps)
        
    mn = np.array([result[k] for k in result]).mean()
                    
    return [result, mn]

def solve_w_pandas(filename):
    ratings = pd.read_csv(filename)
    counted = ratings.groupby('userId').agg({'timestamp': [lambda x: max(x) - min(x)], 'rating': ['count']}).reset_index()
    res = counted[ counted['rating']['count'] > 100 ][['userId', 'timestamp']]
    res.columns = ['userId', 'lifeTime']
    mn = res['lifeTime'].mean()
    return [res, mn]


filename = 'ml-latest-small/ratings.csv'
if is_sorted_by_userid(filename):
    res, mn = solve_wo_pandas(filename)
    cnt = 0
    mn = np.array([res[k] for k in res]).mean()
    print(f"Решение без использования Pandas, среднее: {int(mn)}.")
    for user_id, life_time in res.items():
        print(user_id, life_time)
        cnt += 1
        if cnt == 10:
            break

res_p, mn = solve_w_pandas(filename)
#mn = np.array([res_p[k] for k in res_p]).mean()
print(f"\nРешение с использованием Pandas, среднее: {int(mn)}.")
res_p.head(10)

Решение без использования Pandas, среднее: 40080507.
4 203560
8 85187
15 471393496
17 8053
19 5282
21 1365432
22 91491
23 18342129
26 20267261
30 115852276

Решение с использованием Pandas, среднее: 40080507.


,userId,lifeTime
3,4,203560
7,8,85187
14,15,471393496
16,17,8053
18,19,5282
20,21,1365432
21,22,91491
22,23,18342129
25,26,20267261
29,30,115852276


**Задание 2**  
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия).
```python
rzd = pd.DataFrame({
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    })
auto = pd.DataFrame({
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    })
air = pd.DataFrame({
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    })
client_base = pd.DataFrame({
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    })
```
Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента  

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [33]:
from IPython.display import display, HTML

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

table1 = rzd.merge(auto, how="outer").merge(air, how="outer").fillna("0")
table2 = table1.merge(client_base, how="outer").fillna("0")

display(HTML(table1.to_html()))
display(HTML(table2.to_html()))

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093,0,0
1,112,2810,0,0
2,113,10283,57483,0
3,114,5774,83,0
4,115,981,912,81
5,116,0,4834,4
6,117,0,98,13
7,118,0,0,173


,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093,0,0,Комсомольская 4
1,112,2810,0,0,Энтузиастов 8а
2,113,10283,57483,0,Левобережная 1а
3,114,5774,83,0,Мира 14
4,115,981,912,81,ЗЖБИиДК 1
5,116,0,4834,4,Строителей 18
6,117,0,98,13,Панфиловская 33
7,118,0,0,173,Мастеркова 4


**Задание 3**  
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

**Решение**  
1. Если есть координаты пользователей, то должны быть и координаты мест, где они совершают покупки. Соответсвенно, связать можно по координатам точек продаж и user_id.
2. Если у посетителя различные координаты, значит, он побывал в разных точках или отделах точек продаж. Тогда их можно связать с категорией продаваемых товаров.
3. Имея наборы координат по разным пользователям можно выявить закономерности между тем какие отделы посещают покупатели, при какой последовательности посещения было совершено больше покупок. Затем вывесить баннеры какие отделы еще рекомендуется посетить. Так покупатели быстрее смогут найти интересные им отделы, а продавцы - потенциальных покупателей.